<a href="https://colab.research.google.com/github/Nileshsri2022/genai/blob/main/Copy_of_LLAMA_3_1_70B_RAG_Doc_QA_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install langchain-chroma==0.1.4 langchain==0.3.13 langchain-community==0.3.13 langchain-text-splitters==0.3.4 langchain-groq==0.2.2 transformers==4.47.1 sentence-transformers==3.3.1 unstructured==0.16.11 unstructured[pdf]==0.16.11

In [4]:
import os
#like pypdf loader but here use
from langchain.document_loaders import UnstructuredFileLoader
#chunk your document
from langchain_text_splitters import CharacterTextSplitter
#different type of embeddings like openai but in openai there is charge free credit
#not there API_KEY so use HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
#chromadb is depricated
from langchain_chroma import Chroma
# to load the model
from langchain_groq import ChatGroq
#document QandA
from langchain.chains import RetrievalQA

In [5]:
os.environ["GROQ_API_KEY"] = "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"

In [8]:
# laoding the document
loader = UnstructuredFileLoader("/content/attension_is_all_you_need.pdf")
documents = loader.load()

In [9]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [11]:
texts = text_splitter.split_documents(documents)

In [12]:
type(texts)

list

In [13]:
len(texts)

26

In [16]:
#texts[:3]

In [17]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-17-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-17-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
persist_directory = "doc_db"

In [19]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [20]:
# retriever
retriever = vectordb.as_retriever()

In [21]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [22]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [23]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})

In [24]:
print(response)

{'query': 'What is the model architecture discussed in this paper?', 'result': 'The model architecture discussed in this paper is the Transformer model, which consists of an encoder and a decoder. Both the encoder and decoder are composed of a stack of 6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization to facilitate the flow of information.', 'source_documents': [Document(metadata={'source': '/content/attension_is_all_you_need.pdf'}, page_content='2\n\nFigure 1: The Transformer - model architecture.\n\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.\n\n3.1 En

In [25]:
print(response["result"])

The model architecture discussed in this paper is the Transformer model, which consists of an encoder and a decoder. Both the encoder and decoder are composed of a stack of 6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization to facilitate the flow of information.


In [26]:
print(response["source_documents"][0].metadata["source"])

/content/attension_is_all_you_need.pdf


In [27]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

The model architecture discussed in this paper is the Transformer model, which consists of an encoder and a decoder. Both the encoder and decoder are composed of a stack of 6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs multi-head attention over the output of the encoder stack. The model uses residual connections and layer normalization to facilitate the flow of information.
******************************
Source Document: /content/attension_is_all_you_need.pdf
